In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymatgen.core import Structure 
from nebula.generate import InterstitialNEB

In [3]:
supercell = Structure.from_file('POSCAR')*[3,3,2]
supercell.lattice.abc

(13.678227857180952, 13.678227857180952, 14.886216052355561)

In [4]:
ineb = InterstitialNEB()
possibilities = ineb.get_interstitial_neb_mappings(
    supercell = supercell,
    desired_interstitial_specie='Ag',
    neb_distance_cutoff = 4,
)

Generating DefectEntry objects: 100.0%|██████████| [00:00,  128.95it/s]



2 interstitial(s) found from doped:
 - ['Ag_i_C3v_Ag2.67_0', 'Ag_i_C3v_Ag2.98_0']
            

3 pathways found:
                 site1              site2 direction
0    Ag_i_C3v_Ag2.67_0  Ag_i_C3v_Ag2.67_0         c
2    Ag_i_C3v_Ag2.67_0  Ag_i_C3v_Ag2.98_0         c
108  Ag_i_C3v_Ag2.98_0  Ag_i_C3v_Ag2.98_0         c
              


In [6]:
from nebula.plotting import NEBulaPlotter

for i,path in possibilities.items():
    nebp = NEBulaPlotter()
    images = ineb.neb_create(path['init_structure'],path['final_structure'])
    nebp.save_neb_as_poscar(images,filename=f'poscar_{i}.vasp',mobile_species='Ag')

In [8]:
!open poscar_*.vasp 

In [ ]:
from chgnet.model import CHGNetCalculator
from nebula.relax import NEBulaRelax

cnc = CHGNetCalculator()
nebrelax = NEBulaRelax(asecalculator = cnc)
energy, fstruct = nebrelax.relax(supercell)

In [ ]:
ineb = InterstitialNEB()
possibilities = ineb.get_interstitial_neb_mappings(
    supercell = fstruct,
    desired_interstitial_specie='Ag',
    neb_distance_cutoff = 4,
)

In [ ]:
ims = []
for poss in possibilities.values():
    _images = nebrelax.nebrun(poss['init_structure'],poss['final_structure'],relax_endpoints=True) #
    ims.append(_images)

In [ ]:
from nebula.plotting import NEBulaPlotter
import seaborn as sns 
import matplotlib.pyplot as plt 

sns.set_style('darkgrid')
sns.set_palette('pastel')

fig,ax = plt.subplots(figsize=(6,6),dpi=100)
nebplot = NEBulaPlotter()

for i,neb in enumerate(ims):
    nebplot.nebanalysis(neb)
    ax = nebplot.plot_neb(ax=ax)

#### 1. Initialise INeb and relax using CHGNet (optional)

In [ ]:
ineb = INeb()
energy,structure = ineb.chgnet_relax(structure=struct,**{'steps':1,'fmax':0.1,'relax_model':'MDMin'})
energy

### 2. generate interstitial mappings using Doped (https://github.com/SMTG-Bham/doped)

In [ ]:
mappings = ineb.get_interstitial_neb_mappings(structure,relax_with_chgnet=False)

### 3. generate interstitialcy mappings based on the interstitial pathways found

In [ ]:
mappings_intcy = ineb.get_interstitialcy_neb_mappings(structure,passthrough_search_radius=3)

In [ ]:
'''
save the interstitial images as POSCAR files to run with VASP 
'''
# ineb.save_interstitial_VASP(location='interstitial',nimages=5)

In [ ]:
'''
save the interstitialcy images as POSCAR files to run with VASP 
'''
# ineb.save_interstitialcy_VASP(location='interstitialcy',nimages=5)

### 4. you can also run a simple NEB using CHGNet (https://chgnet.lbl.gov) as the calculator through ASE (https://wiki.fysik.dtu.dk/ase/). 

#### 4a Interstitial NEB

In [ ]:
'''relax the end points first '''
_,ii = ineb.chgnet_relax(mappings[2592]['init_structure'],fmax=0.5)
_,ff = ineb.chgnet_relax(mappings[2592]['final_structure'],fmax=0.5)

In [ ]:
'''now the NEB calcualtion'''
neb = ineb.chgnet_nebrun(ii,ff,nimages=3,fmax=0.5)

In [ ]:
'''and now plot the result'''
try:
    import seaborn as sns
    sns.set_style('darkgrid')
    sns.set_palette('pastel')
except Exception:
    pass 
import matplotlib.pyplot as plt 
from ase.neb import NEBTools
fig,ax = plt.subplots(figsize=(4,4),dpi=100)
nebtools = NEBTools(neb)
Ef, dE = nebtools.get_barrier()
fit = nebtools.get_fit()
ax.plot(fit[2], fit[3])
ax.plot(fit[0], fit[1], **{'marker': 'o',
                           'linestyle': 'None',
                           'color': 'grey',
                           'markerfacecolor': 'white',
                           'markersize': 5})
ax.set_ylabel('energy (eV)')
ax.set_xlabel('path coordinate (arb. units)')
plt.show()

#### 4b Interstitialcy NEB

In [ ]:
'''relax the end points first '''
_,ii = ineb.chgnet_relax(mappings_intcy[2592][0]['init_structure'],fmax=0.5)
_,ff = ineb.chgnet_relax(mappings_intcy[2592][0]['final_structure'],fmax=0.5)

In [ ]:
'''now the NEB calcualtion'''
neb_intcy = ineb.chgnet_nebrun(ii,ff,nimages=3,fmax=0.5)

In [ ]:
'''and now plot the result'''
try:
    import seaborn as sns
    sns.set_style('darkgrid')
    sns.set_palette('pastel')
except Exception:
    pass 
import matplotlib.pyplot as plt 
from ase.neb import NEBTools
fig,ax = plt.subplots(figsize=(4,4),dpi=100)
nebtools = NEBTools(neb_intcy)
Ef, dE = nebtools.get_barrier()
fit = nebtools.get_fit()
ax.plot(fit[2], fit[3])
ax.plot(fit[0], fit[1], **{'marker': 'o',
                           'linestyle': 'None',
                           'color': 'grey',
                           'markerfacecolor': 'white',
                           'markersize': 5})
ax.set_ylabel('energy (eV)')
ax.set_xlabel('path coordinate (arb. units)')
plt.show()

#### 4c plot them together

In [ ]:
'''and now plot the result'''
try:
    import seaborn as sns
    sns.set_style('darkgrid')
    sns.set_palette('pastel')
except Exception:
    pass 
import matplotlib.pyplot as plt 
from ase.neb import NEBTools

labels = {0:'interstitial',1:'interstitialcy'}

fig,ax = plt.subplots(figsize=(4,4),dpi=100)
for i,calc in enumerate([neb,neb_intcy]):
    nebtools = NEBTools(calc)
    Ef, dE = nebtools.get_barrier()
    fit = nebtools.get_fit()
    ax.plot(fit[2], fit[3],label=labels[i])
    ax.plot(fit[0], fit[1], **{'marker': 'o',
                               'linestyle': 'None',
                               'color': 'grey',
                               'markerfacecolor': 'white',
                               'markersize': 5})
ax.set_ylabel('energy (eV)')
ax.set_xlabel('path coordinate (arb. units)')
ax.legend(edgecolor='black')
plt.show()